# Template Deployment Script

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import azureml.core
import os
import pickle
import catboost
import sys
import json

from azureml.core import Workspace, Webservice
from azureml.core.model import Model

%load_ext autoreload
%autoreload 2

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
###########################################################
###########################################################

# What github repository are you using?

project = "github repo name"

# Set the service name before continuing

serviceName = ''

# What is the name of the model as used in the deployment json and in score.py?

model_name = ""

# What subscription is this in

deploymentEnvironment = ''



###########################################################
###########################################################

In [ ]:
### CHANGE TO YOUR WORKSPACE

ws = Workspace(
    workspace_name=workspace_name, 
    subscription_id=subscription_id, 
    resource_group=resource_group)
display(ws)


print('Referencing:', deploymentEnvironment)

In [ ]:
%load_ext autoreload
%autoreload 2
from datascience_core.codecandy import jupyter as candy


## Point to your workspace

## Load in the model and register in the service - note if you do this twice it creates a second version

In [ ]:
if False:
    model = Model.register(workspace = ws,
                             model_path =model_path,
                             model_name = model_name,
                             description = serviceDescription,)

## Check the model exists and point to it for deployment later

In [ ]:
model=Model(ws, model_name)
#model.download(target_dir=os.getcwd(), exist_ok=True) # download the model if you wish
model

## Create and write the score.py script that deploys within the service

In [ ]:
%%writefile score.py
import json
import numpy as np
import pandas as pd
import os
import pickle
import copy
from random import random
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
import catboost
from pandas import json_normalize  
from azureml.core.model import Model
import datetime
import logging 

def log_it(service,message,severity):
    lsSeverity = ['PASS', 'WARNING', 'ERROR', 'PAYLOAD', 'RESULT']
    if severity in lsSeverity:
        try:
            json.loads(message)
            message = '{' + '"Service":"{}", "Severity":"{}", "Message":{}'.format(service,severity,message) + '}'
        except:
            message = '{' + '"Service":"{}", "Severity":"{}", "Message":{}'.format(service,severity,message) + '}'
        print(message)
    else:
        message = '{' + '"Service":"{}", "Severity":"{}", "Message":{}'\
            .format(service, 'LOGGING FAIL', '{' + '"Event": "Severity level not recognised, refer to score.py"' + '}') \
                  + '}'
        print(message)



def init():
    # Init is only run once when the docker container is created so don't try to set any variables here that wil be passed as json later
    global model
    # Set the default model CHECK THIS IS RIGHT IT MUST EXIST
    model_path = Model.get_model_path("modelName")
    model = joblib.load(model_path)
    
    
def run(raw_data):
    service = 'ServiceName'
    try:

        messageStart = '{' + '"someId": "{}", "Event": "{}"'.format(applicationids, 'scoring initiated') + '}'
        log_it(service, messageStart,'PASS')
        log_it(service, raw_data, 'PAYLOAD')

       
         # Pull predictors from the model
        predictors = model.feature_names_

         # Fill the predictors I don't have with 0
        for col in predictors:
            if col not in test_data.columns:
                test_data[col] = 0
                messageColMissing = '{' + '"someId": "{}", "Event": "{}", "Item": "{}"'.format(applicationids, 'MissingCol', col) + '}'
                log_it(service, messageColMissing,'WARNING')

        # Loop through each json payload and find a result object

            thisConf =  model.predict_proba(test_data)
            message = '{' + '"someId": "{}", "Confidence": "{}", "Result": "{}"'.format(applicationids[i],thisConf[0], outcome) + '}'
            log_it(service, message,'RESULT')

         # Create a json object to return
      
        messageEnd = '{' + '"someId":"{}", "Event":"{}"'.format(applicationids, 'Scoring Completed') + '}'
        log_it(service,messageEnd,'PASS')

        return y_hat
    except Exception as e:
        try:
            
            messageException = '{' + '"Exception":"{}", "someId":"{}", "RawData":{}'.format(e, application_ids, raw_data) + '}'
            log_it(service, messageException, "ERROR")
            y_hat_safe = []
            for appid in applicationids:
                y_hat_safe.append({"someId":str(appid), "isReject":True})
            return y_hat_safe
        except Exception as e2:
            # this block will only get activated if there is an error with parsing the json at the start of the modelling process
            messageException = '{' + '"Exception":"{}", "someId":"{}", "RawData":{}'.format(e, np.nan, raw_data) + '}'
            log_it(service, messageException, "ERROR")
            y_hat_safe = {"someId":np.nan, "isReject":True}
            return y_hat_safe



# Deploy the service

## Create and check you enviroment file

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn==0.19.2")
myenv.add_conda_package("xgboost==0.90")
myenv.add_conda_package("catboost==0.22")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

In [ ]:
#service.delete()

In [ ]:
display(serviceName)
display(ws)

In [ ]:
service = Webservice(ws, serviceName)
service

## Specify the aci configuration

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=2, 
                                               #tags={"data": "N/A",  "method" : "sklearn"}, 
                                               description=serviceDescription,
                                               auth_enabled=True,
                                               enable_app_insights=True) 

## Deploy the service for the first time or update the same endpoint

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.image import Image

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="myenv.yml")


# service = Model.deploy(workspace=ws, 
#                        name=serviceName, 
#                        models=[model],
#                        #models=[model,model_pp],
#                        inference_config=inference_config,
#                        deployment_config=aciconfig,
#                        overwrite=True)

# service.wait_for_deployment(show_output=True)

############################################################
### Use this to update the service
############################################################
#service = Webservice(ws, serviceName)
service.update(
    inference_config=inference_config, 
    enable_app_insights=True, 
    ssl_enabled=True, 
    ssl_cert_pem_file="cert.pem", 
    ssl_key_pem_file="key.pem", 
    ssl_cname="",)
#im = Image(ws,name='e',tags=1)
#service.update(image=im)

service.wait_for_deployment(show_output=True)

############################################################
### If you want to point to the service use this
### useful if restart the kernal :)
############################################################
#service = Webservice(ws, '')


print("Successfully deployed the ACI service")
print('Service status:',service.state)
key, secondary = service.get_keys()
display(service.scoring_uri)
display(key)
display(secondary)
service

In [ ]:
#service.delete()

In [ ]:
key, secondary = service.get_keys()
display(service.scoring_uri)
display(key)
display(secondary)
service